In [1]:
print(sc.version)

2.4.0


In [8]:
# Get Project Type that Receive most donations
dfProjectType=spark.sql("SELECT ProjectsHiveTable.`Project Type`, \
             SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total \
             from ProjectsHiveTable JOIN DonationsHiveTable ON \
             ProjectsHiveTable.`Project ID` = DonationsHiveTable.`Project ID` \
             GROUP BY ProjectsHiveTable.`Project Type`")
dfProjectType.show()

+--------------------+------------+
|        Project Type|       Total|
+--------------------+------------+
|         Teacher-Led|276414228.33|
|Professional Deve...|  2491235.29|
|         Student-Led|  2713256.98|
+--------------------+------------+



In [6]:
# Correlation of Teachers and Non Teachers Donations - Teachers Data
dfTeachersData=spark.sql("SELECT SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total, \
              'IsTeacher' AS DonorType \
              FROM TeachersHiveTable JOIN DonationsHiveTable ON \
              TeachersHiveTable.`Teacher ID` = DonationsHiveTable.`Donor ID` \
              GROUP BY DonorType")
dfTeachersData.show()

+-----------+---------+
|      Total|DonorType|
+-----------+---------+
|58141038.72|IsTeacher|
+-----------+---------+



In [3]:
# Correlation of Teachers and Non Teachers Donations - Non Teachers Data
dfNotTeachersData=spark.sql("SELECT SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total, \
              'IsNotTeacher' AS DonorType \
               FROM DonationsHiveTable \
               WHERE NOT EXISTS \
               (SELECT 1 FROM TeachersHiveTable WHERE DonationsHiveTable.`Donor ID` = TeachersHiveTable.`Teacher ID`) \
               GROUP BY DonorType")
dfNotTeachersData.show()

+------------+------------+
|       Total|   DonorType|
+------------+------------+
|226267204.56|IsNotTeacher|
+------------+------------+



In [19]:
# Correlation of Teachers and Non Teachers Donations - Final
dfFinal = dfTeachersData.unionAll(dfNotTeachersData);
dfFinal.show()

+------------+------------+
|       Total|   DonorType|
+------------+------------+
| 58141038.72|   IsTeacher|
|226267204.56|IsNotTeacher|
+------------+------------+



In [21]:
# Correlation of Teachers and Non Teachers Donations - Teachers Data
df=spark.sql("SELECT SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total, \
              DonorsHiveTable.`Donor is Teacher` \
              FROM DonationsHiveTable JOIN DonorsHiveTable ON \
              DonorsHiveTable.`Donor ID` = DonationsHiveTable.`Donor ID` \
              GROUP BY DonorsHiveTable.`Donor is Teacher`")
df.show()

+------------+----------------+
|       Total|Donor is Teacher|
+------------+----------------+
|223064734.47|              No|
| 61006942.31|             Yes|
+------------+----------------+



In [20]:
# Correlation of Males and Females Teachers Donations
dfTeachersData=spark.sql("SELECT SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total, \
              TeachersHiveTable.`Teacher Prefix` \
              FROM DonationsHiveTable JOIN TeachersHiveTable ON \
              TeachersHiveTable.`Teacher ID` = DonationsHiveTable.`Donor ID` \
              GROUP BY TeachersHiveTable.`Teacher Prefix`")
dfTeachersData.show()

+-----------+--------------+
|      Total|Teacher Prefix|
+-----------+--------------+
|20839892.59|           Ms.|
|  703063.40|       Teacher|
|    2271.07|          null|
|10140481.41|           Mr.|
|26423783.32|          Mrs.|
|   25687.58|           Dr.|
|    5859.35|           Mx.|
+-----------+--------------+



In [24]:
# State with most make donations
df=spark.sql("SELECT SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total, \
              DonorsHiveTable.`Donor State` \
              FROM DonationsHiveTable JOIN DonorsHiveTable ON \
              DonorsHiveTable.`Donor ID` = DonationsHiveTable.`Donor ID` \
              GROUP BY DonorsHiveTable.`Donor State`")
df.show(df.count(), False)

+-----------+--------------------+
|Total      |Donor State         |
+-----------+--------------------+
|2755841.98 |Utah                |
|1397935.03 |Hawaii              |
|3604963.49 |Minnesota           |
|5625645.80 |Ohio                |
|1432189.74 |Arkansas            |
|3965820.04 |Oregon              |
|17392010.06|Texas               |
|349755.14  |North Dakota        |
|8719832.46 |Pennsylvania        |
|4546151.53 |Connecticut         |
|670643.94  |Nebraska            |
|384306.38  |Vermont             |
|2348658.13 |Nevada              |
|7437949.12 |Washington          |
|14838197.91|Illinois            |
|4106344.41 |Oklahoma            |
|7993566.28 |other               |
|1811878.20 |District of Columbia|
|915053.54  |Delaware            |
|669350.43  |Alaska              |
|1089004.25 |New Mexico          |
|812536.38  |West Virginia       |
|4035165.36 |Missouri            |
|802152.86  |Rhode Island        |
|7719309.94 |Georgia             |
|687270.29  |Montana

In [28]:
# State with most received donations
df=spark.sql("SELECT SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total, \
              SchoolsHiveTable.`School State` \
              FROM DonationsHiveTable \
              JOIN ProjectsHiveTable ON \
              DonationsHiveTable.`Project ID` = ProjectsHiveTable.`Project ID` \
              JOIN SchoolsHiveTable ON \
              SchoolsHiveTable.`School ID` = ProjectsHiveTable.`School ID` \
              GROUP BY SchoolsHiveTable.`School State`")
df.show(df.count(), False)

+-----------+--------------------+
|Total      |School State        |
+-----------+--------------------+
|3108568.37 |Utah                |
|2019575.50 |Hawaii              |
|3414589.27 |Minnesota           |
|5720818.77 |Ohio                |
|4075159.99 |Oregon              |
|1784650.09 |Arkansas            |
|17539223.58|Texas               |
|356774.25  |North Dakota        |
|8333432.70 |Pennsylvania        |
|4615298.66 |Connecticut         |
|394128.26  |Vermont             |
|646896.23  |Nebraska            |
|2773494.78 |Nevada              |
|6441378.47 |Washington          |
|15542406.18|Illinois            |
|5231957.03 |Oklahoma            |
|2129021.31 |District of Columbia|
|919433.67  |Delaware            |
|899257.67  |Alaska              |
|1288396.57 |New Mexico          |
|1158237.59 |West Virginia       |
|4894340.48 |Missouri            |
|1113406.14 |Rhode Island        |
|8297850.00 |Georgia             |
|815422.36  |Montana             |
|5750947.28 |Virgini

In [29]:
# School Type with most received donations
df=spark.sql("SELECT SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total, \
              SchoolsHiveTable.`School Metro Type` \
              FROM DonationsHiveTable \
              JOIN ProjectsHiveTable ON \
              DonationsHiveTable.`Project ID` = ProjectsHiveTable.`Project ID` \
              JOIN SchoolsHiveTable ON \
              SchoolsHiveTable.`School ID` = ProjectsHiveTable.`School ID` \
              GROUP BY SchoolsHiveTable.`School Metro Type`")
df.show(df.count(), False)

+------------+-----------------+
|Total       |School Metro Type|
+------------+-----------------+
|22271045.27 |unknown          |
|82873104.89 |suburban         |
|11738111.95 |town             |
|26110253.26 |rural            |
|138624792.87|urban            |
+------------+-----------------+



In [10]:
# Project Subject Category that Receive most donations - NOT REALLY
df=spark.sql("SELECT ProjectsHiveTable.`Project Subject Category Tree`, \
             SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total \
             from ProjectsHiveTable JOIN DonationsHiveTable ON \
             ProjectsHiveTable.`Project ID` = DonationsHiveTable.`Project ID` \
             WHERE ProjectsHiveTable.`Project Subject Category Tree` = 'Special Needs' \
             GROUP BY ProjectsHiveTable.`Project Subject Category Tree`")
df.show()

+-----------------------------+----------+
|Project Subject Category Tree|     Total|
+-----------------------------+----------+
|                Special Needs|1724975.64|
+-----------------------------+----------+



In [1]:
# Project Resource category - Books vs Techonology - that Receive most donations
df=spark.sql("SELECT ProjectsHiveTable.`Project Resource Category`, \
             SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total \
             from ProjectsHiveTable \
             JOIN DonationsHiveTable ON \
             ProjectsHiveTable.`Project ID` = DonationsHiveTable.`Project ID` \
             WHERE ProjectsHiveTable.`Project Resource Category` = 'Technology' \
             OR ProjectsHiveTable.`Project Resource Category` = 'Books' \
             GROUP BY ProjectsHiveTable.`Project Resource Category`")
df.show()

+-------------------------+-----------+
|Project Resource Category|      Total|
+-------------------------+-----------+
|                    Books| 9873985.77|
|               Technology|23669675.03|
+-------------------------+-----------+



In [2]:
# Get Total of Donations
df=spark.sql("SELECT 'TotalDonations', \
             SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total \
             from DonationsHiveTable \
             GROUP BY TotalDonations")
df.show()

+--------------+------------+
|TotalDonations|       Total|
+--------------+------------+
|TotalDonations|284408243.28|
+--------------+------------+



In [9]:
# Get Donations by Date
df=spark.sql("SELECT SUBSTRING(DonationsHiveTable.`Donation Received Date`, 0,10) AS DonationDate, \
             SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total \
             from DonationsHiveTable \
             GROUP BY DonationDate \
             ORDER BY DonationDate DESC")
df.show()

+------------+---------+
|DonationDate|    Total|
+------------+---------+
|  2018-05-09| 98526.99|
|  2018-05-08|441211.77|
|  2018-05-07|153675.41|
|  2018-05-06| 67549.57|
|  2018-05-05| 55167.27|
|  2018-05-04| 92190.50|
|  2018-05-03|137752.54|
|  2018-05-02|173867.18|
|  2018-05-01|286805.07|
|  2018-04-30|190048.95|
|  2018-04-29|138624.27|
|  2018-04-28|154656.43|
|  2018-04-27|196513.06|
|  2018-04-26|178205.34|
|  2018-04-25|251247.40|
|  2018-04-24|349875.78|
|  2018-04-23|275768.65|
|  2018-04-22|218696.37|
|  2018-04-21|166899.77|
|  2018-04-20|217308.53|
+------------+---------+
only showing top 20 rows



In [10]:
# Get Donations by Year
df=spark.sql("SELECT SUBSTRING(DonationsHiveTable.`Donation Received Date`, 0,4) AS DonationDate, \
             SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total \
             from DonationsHiveTable \
             GROUP BY DonationDate \
             ORDER BY DonationDate DESC")
df.show()

+------------+-----------+
|DonationDate|      Total|
+------------+-----------+
|        2018|26481196.48|
|        2017|72872130.78|
|        2016|62652231.24|
|        2015|50488354.49|
|        2014|41461266.85|
|        2013|30428220.80|
|        2012|   24842.64|
+------------+-----------+



In [11]:
# Get Donations by Month/Year
df=spark.sql("SELECT SUBSTRING(DonationsHiveTable.`Donation Received Date`, 0,7) AS DonationDate, \
             SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total \
             from DonationsHiveTable \
             GROUP BY DonationDate \
             ORDER BY DonationDate DESC")
df.show()

+------------+----------+
|DonationDate|     Total|
+------------+----------+
|     2018-05|1506746.30|
|     2018-04|6313794.97|
|     2018-03|5757581.91|
|     2018-02|5094398.32|
|     2018-01|7808674.98|
|     2017-12|8466878.98|
|     2017-11|6887667.01|
|     2017-10|7077864.75|
|     2017-09|7588821.83|
|     2017-08|9043762.30|
|     2017-07|4826303.21|
|     2017-06|2983810.52|
|     2017-05|3721099.67|
|     2017-04|3418379.81|
|     2017-03|7160669.35|
|     2017-02|5864177.41|
|     2017-01|5832695.94|
|     2016-12|8152607.93|
|     2016-11|6271338.86|
|     2016-10|5478470.10|
+------------+----------+
only showing top 20 rows



In [13]:
# Test Create table and export to csv
# Get Donations by Year
df=spark.sql("SELECT SUBSTRING(DonationsHiveTable.`Donation Received Date`, 0,4) AS DonationDate, \
             SUM(CAST(DonationsHiveTable.`Donation Amount` AS DECIMAL(10,2))) AS Total \
             from DonationsHiveTable \
             GROUP BY DonationDate \
             ORDER BY DonationDate DESC")
df.createOrReplaceTempView("TestTempTable")
spark.sql("create table TestTempTable as select * from TestTempTable")

DataFrame[]